# Advent of Code 2021

This solution (Jupyter notebook; coconut 1.5.0 on python 3.7.11) by kannix68, @ 2021-12.  \
Using anaconda distro, conda v4.10.3, and coconut language. installation on MacOS v10.14.6 "Mojave".

In [ ]:
import sys
import logging
import itertools
import re

from collections import defaultdict

import numpy as np
import pandas as pd

import pylib.aochelper as aoc
#from pylib.aochelper import map_list as mapl
#from pylib.aochelper import filter_list as filterl

f"Python version: {sys.version}" |> print
f"Version info: {sys.version_info}" |> print

log = aoc.getLogger(__name__)
f"Initial log-level={aoc.getLogLevelName(log.getEffectiveLevel())}." |> print

## Problem domain code

### Day 1: Sonar Sweep

In [ ]:
"Day 1" |> print

tests = """
199
200
208
210
200
207
240
269
260
263""".strip()

In [ ]:
def solve_d01pt1(inp):
  """Solve Day 1 part 1."""
  inp = inp |> .split() |> map$(int)
  outp = pd.Series(inp).diff()[1:].astype(int).tolist()
  outp = outp |> filter$(it -> it > 0)
  return outp |> list |> len

In [ ]:
expected = 7
result = solve_d01pt1(tests)
aoc.assert_msg("test solve day 1 part 1", result == expected) 

In [ ]:
ins = aoc.read_file_to_str("./in/day01.in")
out = solve_d01pt1(ins)
f"day 1 part 1 output: {out}" |> print

In [ ]:
"Day 1 part 2" |> print

In [ ]:
def solve_d01pt2(inp):
  """Solve Day 1 part 2."""
  inp = inp |> .split() |> map$(int)
  outp = pd.Series(inp).rolling(3).sum().diff()[3:].astype(int).tolist()
  return outp |> filter$(-> _ > 0) |> list |> len

In [ ]:
expected = 5
result = solve_d01pt2(tests)
aoc.assert_msg("test solve day 1 part 2", result == expected) 

In [ ]:
out = solve_d01pt2(ins)
f"day 1 part 2 output: {out}" |> print

## Day 2: Dive!

In [ ]:
"Day 2" |> print

In [ ]:
def iterate(pos, cmd):
  HPOS, DEPTH = 0, 1
  direct, val = cmd.split(" ")
  val = int(val)
  pos = pos.copy()
  case direct:
    match "forward":
      pos[HPOS] += val
    match "down":
      pos[DEPTH] += val
    match "up":
      pos[DEPTH] -= val
  return pos

def solve_d02pt1(inp):
  pos = [0, 0]
  for cmd in inp.splitlines():
    pos = iterate(pos, cmd)
  return pos |> reduce$(*)

tests = """
forward 5
down 5
forward 8
up 3
down 8
forward 2""".strip()

expected = 150
result = solve_d02pt1(tests)
aoc.assert_msg("test solve day 1 part 1", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day02.in")
out = solve_d02pt1(ins)
f"day 2 part 1 output: {out}" |> print

In [ ]:
"Day 2 part 2" |> print

In [ ]:
def iterate2(pos, cmd):
  HPOS, DEPTH, AIM = 0, 1, 2
  direct, val = cmd.split(" ")
  val = int(val)
  pos = pos.copy()
  case direct:
    match "forward":
      pos[HPOS] += val
      pos[DEPTH] += pos[AIM] * val
    match "down":
      pos[AIM] += val
    match "up":
      pos[AIM] -= val
  return pos

def solve_d02pt2(inp):
  pos = [0, 0, 0]
  for cmd in inp.splitlines():
    pos = iterate2(pos, cmd)
  return pos[0:2] |> reduce$(*)

expected = 900
result = solve_d02pt2(tests)
aoc.assert_msg("test solve day 1 part 2", result == expected)

In [ ]:
# `ins` remains the same
out = solve_d02pt2(ins)
f"day 2 part 2 output: {out}" |> print

## Day 5: Hydrothermal Venture

In [ ]:
"Day 5" |> print

In [ ]:
tests = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
""".strip()

In [ ]:
def solve_d05pt1(ins):
  lines = ins.splitlines() |> map$(-> _.split(" -> ")) |> list
  for lnidx in range(len(lines)):
    for ptidx in range(2):
      lines[lnidx][ptidx] = lines[lnidx][ptidx].split(",") |> map$(int) |> list
  log.trace(lines)
  grid = defaultdict(int)
  for line in lines:
    ptst, pten = line
    if ptst[0] == pten[0]:  # horiz
      log.trace(f"horiz line {line}")
      x = ptst[0]
      for y in range(min(ptst[1], pten[1]), max(ptst[1], pten[1])+1):
        grid[tuple([x,y])] += 1
    elif ptst[1] == pten[1]:  # vert
      log.trace(f"vert line {line}")
      y = ptst[1]
      for x in range(min(ptst[0], pten[0]), max(ptst[0], pten[0])+1):
        grid[tuple([x,y])] += 1
    else:
      pass
  log.trace(grid)
  intersects = grid.values() |> filter$(-> _ > 1) |> list |> len
  log.debug(f"intersects-#={intersects}")
  return intersects

expected = 5
log.setLevel(logging.DEBUG)
result = solve_d05pt1(tests)
aoc.assert_msg("test solve day 5 part 1", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day05.in")
out = solve_d05pt1(ins)
f"day 2 part 1 output: {out}" |> print

In [ ]:
"Day 5 part 2" |> print

In [ ]:
def scan_grid(lines):
  grid = defaultdict(int)
  for line in lines:
    ptst, pten = line
    if ptst[0] == pten[0] and ptst[1] == pten[1]:  # point
      grid[tuple(ptst)] += 1
    elif ptst[0] == pten[0]:  # horiz
      log.trace(f"horiz line {line}")
      x = ptst[0]
      for y in range(min(ptst[1], pten[1]), max(ptst[1], pten[1])+1):
        grid[tuple([x,y])] += 1
    elif ptst[1] == pten[1]:  # vert
      log.trace(f"vert line {line}")
      y = ptst[1]
      for x in range(min(ptst[0], pten[0]), max(ptst[0], pten[0])+1):
        grid[tuple([x,y])] += 1
    else:  # diagonal
      line = sorted(line, key=lambda it: it[0])
      ptst, pten = line
      if ptst[1] > pten[1]:
        ystp = -1
      else:
        ystp = 1
      log.trace(f"diag line {line}")
      y = ptst[1]
      for x in range(ptst[0], pten[0]+1):
        log.trace(f"set diag line-pt {[x,y]}") 
        grid[tuple([x,y])] += 1
        y += ystp
  log.trace(grid)
  return grid
  
def solve_d05pt2(ins):
  lines = ins.splitlines() |> map$(-> _.split(" -> ")) |> list
  for lnidx in range(len(lines)):
    for ptidx in range(2):
      lines[lnidx][ptidx] = lines[lnidx][ptidx].split(",") |> map$(int) |> list
  log.trace(lines)
  grid = scan_grid(lines)
  intersects = grid.values() |> filter$(-> _ > 1) |> list |> len
  log.debug(f"intersects-#={intersects}")
  #log.debug(f"grid={grid}")
  return intersects

expected = 12
log.setLevel(logging.INFO)
result = solve_d05pt2(tests)
aoc.assert_msg("test solve day 5 part 1", result == expected)

In [ ]:
out = solve_d05pt2(ins)
f"day 2 part 2 output: {out}" |> print